# Simple Model de train test et split

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
import tensorflow as tf

# Chemins vers les dossiers
train_data_dir = "/content/drive/MyDrive/ColabNotebooks/chest_Xray/train"
val_data_dir = "/content/drive/MyDrive/ColabNotebooks/chest_Xray/val"
test_data_dir = "/content/drive/MyDrive/ColabNotebooks/chest_Xray/test"

# Paramètres du générateur d'images
img_width, img_height = 224, 224
batch_size = 32

# Créer les générateurs d'images
datagen = ImageDataGenerator(rescale=1./255)

# Charger les données d'entraînement et de validation
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Effectuer une validation croisée avec K-Fold
k = 5  # Nombre de folds
kf = KFold(n_splits=k)

# Liste pour stocker les scores de chaque fold
scores = []

for train_index, val_index in kf.split(train_generator.filenames):
    # Diviser les données en ensembles d'entraînement et de validation pour chaque fold
    train_files = [train_generator.filenames[i] for i in train_index]
    val_files = [train_generator.filenames[i] for i in val_index]

    train_datagen_fold = ImageDataGenerator(rescale=1./255)
    train_generator_fold = train_datagen_fold.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        classes=["NORMAL", "PNEUMONIA"],
        shuffle=False,
        subset='training',
        seed=42)

    val_datagen_fold = ImageDataGenerator(rescale=1./255)
    val_generator_fold = val_datagen_fold.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        classes=["NORMAL", "PNEUMONIA"],
        shuffle=False,
        subset='validation',
        seed=42)

    # Créer, entraîner et évaluer le modèle pour chaque fold
model = Sequential()
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(name='recall') ])

model.fit(train_generator_fold, validation_data=val_generator_fold, epochs=40, verbose=1)
score = model.evaluate(val_generator, verbose=0)[1]  # Récupérer le score d'accuracy
scores.append(score)

# Afficher les scores de chaque fold
for i, score in enumerate(scores):
    print(f"Fold {i+1} - Accuracy: {score}")

# Afficher le score moyen
mean_score = np.mean(scores)
print("Mean Accuracy:", mean_score)

# Évaluer le modèle sur les données de test

test_score = model.evaluate(test_generator, verbose=0)[1]
print("Test Accuracy:", test_score)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Epoch 1/40
163/163 [==============================] - 77s 453ms/step - loss: 14.7935 - accuracy: 0.6099 - recall: 0.7466
Epoch 2/40
163/163 [==============================] - 74s 452ms/step - loss: 0.8206 - accuracy: 0.7429 - recall: 1.0000
Epoch 3/40
163/163 [==============================] - 73s 449ms/step - loss: 0.8058 - accuracy: 0.7402 - recall: 0.9964
Epoch 4/40
163/163 [==============================] - 73s 445ms/step - loss: 0.6724 - accuracy: 0.7398 - recall